In [1]:
import keras 

init_model = keras.applications.mobilenet.MobileNet(input_shape=None, 
                                                    alpha=1.0, 
                                                    depth_multiplier=1, 
                                                    dropout=1e-3, 
                                                    weights='imagenet', 
                                                    pooling='max')


/anaconda3/envs/cityCar/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/envs/cityCar/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


17227776/17225924 [==============================] - 3s 0us/step


In [2]:
x = init_model.output
x = keras.layers.Dense(512,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
preds = keras.layers.Dense(163,activation='softmax')(x)
model = keras.models.Model(inputs=init_model.input,outputs=preds)

In [3]:
for layer in model.layers[:-6]:
    layer.trainable=False
for layer in model.layers[-6:]:
    layer.trainable=True

In [4]:
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics = ["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________

In [13]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.8,1.2],
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

test_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        '../data/external/compcars_data/train_image',
        target_size=(224, 224),
        batch_size=16)

validation_generator = test_datagen.flow_from_directory(
        '../data/external/compcars_data/test_image',
        target_size=(224, 224),
        batch_size=16)

csv_logger = keras.callbacks.CSVLogger('training.log')
saver = keras.callbacks.ModelCheckpoint("mobile_net.{epoch:02d}-{val_acc:.2f}.hdf5", monitor='val_acc')

Found 79528 images belonging to 163 classes.
Found 57114 images belonging to 163 classes.


In [14]:
model.fit_generator(train_generator,
                    validation_data=validation_generator,
                    epochs=10,
                    steps_per_epoch=train_generator.n // train_generator.batch_size, 
                    validation_steps=validation_generator.n // validation_generator.batch_size,
                    workers=8,
                    callbacks=[csv_logger, saver],
                    use_multiprocessing=True)

Epoch 1/10
  23/4970 [..............................] - ETA: 1:29:07 - loss: 4.5179 - acc: 0.0380

Process ForkPoolWorker-47:
Process ForkPoolWorker-48:
Process ForkPoolWorker-42:
Process ForkPoolWorker-45:
Process ForkPoolWorker-43:
Process ForkPoolWorker-41:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/envs/cityCar/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/anaconda3/envs/cityCar/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/anaconda3/envs/cityCar/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/anaconda3/envs/cityCar/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/anaconda3/envs/cityCar/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/anaconda3/envs/cityCar/lib/python3.5/multiprocess

KeyboardInterrupt: 

In [6]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                             shear_range=0.1,
                                                             zoom_range=0.1,
                                                             brightness_range=[0.8,1.2],
                                                             horizontal_flip=True)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  horizontal_flip=True)

big_train_generator = train_datagen.flow_from_directory(
        'data/train_images',
        target_size=(224, 224),
        batch_size=2048)

big_val_generator = validation_datagen.flow_from_directory(
        'data/test_images',
        shuffle=False,
        target_size=(224, 224),
        batch_size=2048)

Found 79528 images belonging to 163 classes.
Found 57114 images belonging to 163 classes.


In [ ]:
for i in range(10000):
    X_train, Y_train = big_train_generator.next()
    X_val, Y_val = big_val_generator.next()

  train_generator = train_datagen.flow(X_train,
                                       Y_train,
                                       batch_size=32)

  validation_generator = validation_datagen.flow(X_val,
                                                 Y_val,
                                                 shuffle=False,
                                                 batch_size=32)

  csv_logger = keras.callbacks.CSVLogger('training.log')

  saver = keras.callbacks.ModelCheckpoint("mobile_net.{epoch:02d}-{val_acc:.2f}.hdf5", monitor='val_acc')

  model.fit_generator(train_generator,
                      validation_data=validation_generator,
                      epochs=1,
                      steps_per_epoch=train_generator.n // train_generator.batch_size, 
                      validation_steps=validation_generator.n // validation_generator.batch_size,
                      workers=8,
                      initial_epoch = i - 1,
                      callbacks=[csv_logger, saver],
                      use_multiprocessing=True)

Epoch 0/1
64/64 [==============================] - 37s 574ms/step - loss: 4.8037 - acc: 0.0361 - val_loss: 4.5750 - val_acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 17s 263ms/step - loss: 4.4430 - acc: 0.0337 - val_loss: 4.5081 - val_acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 41s 640ms/step - loss: 4.4766 - acc: 0.0312 - val_loss: 4.6465 - val_acc: 0.0000e+00
